In [ ]:
from sklearn.model_selection import train_test_split
from datasets import load_dataset, load_metric
from huggingface_hub import notebook_login
import re
import os
#os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
#os.environ['TORCH_USE_CUDA_DSA'] = "1"
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, DataCollatorForLanguageModeling
from typing import Any, Dict, List, Optional, Union
import wandb
from torch.utils.data import Dataset
import logging
logger = logging.getLogger(__name__)
import numpy as np
notebook_login()

import sys
sys.path.append('../src')
from src.utils import DataCollatorForCompletionOnlyLM, GoalSolutionDataset, END_KEY, INSTRUCTION_KEY, RESPONSE_KEY_NL

In [2]:
datasets = load_dataset("piqa")

Found cached dataset piqa (C:/Users/shwet/.cache/huggingface/datasets/piqa/plain_text/1.1.0/6c611c1a9bf220943c4174e117d3b660859665baf1d43156230116185312d011)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
datasets.keys()

dict_keys(['train', 'test', 'validation'])

In [3]:
datasets["train"][0]["goal"]

"When boiling butter, when it's ready, you can"

In [4]:
def preprocess_intents_json(segment):
    
    preprocessed_data = []
    
    for data in datasets[segment]:
        preprocessed_data.append(f"Goal: {data['goal']}\n")
        if data['label']=='1':
            preprocessed_data.append(f"Solution: {data['sol2']}\n")
        else:
            preprocessed_data.append(f"Solution: {data['sol1']}\n")
    
    return "".join(preprocessed_data)

def save_preprocessed_data(preprocessed_data, output_file):
    with open(output_file, "w") as f:
        f.write(preprocessed_data)


output_file = f"./data/data_train.txt"
preprocessed_data = preprocess_intents_json("train")
preprocessed_data = re.sub(r'\n+', '\n', preprocessed_data).strip() 
save_preprocessed_data(preprocessed_data, output_file)

output_file = f"./data/data_validation.txt"
preprocessed_data = preprocess_intents_json("validation")
preprocessed_data = re.sub(r'\n+', '\n', preprocessed_data).strip() 
save_preprocessed_data(preprocessed_data, output_file)

output_file = f"./data/data_test.txt"
preprocessed_data = preprocess_intents_json("test")
preprocessed_data = re.sub(r'\n+', '\n', preprocessed_data).strip() 
save_preprocessed_data(preprocessed_data, output_file)

## Preprocessing Data

In [5]:
train_file_path = "./data/data_train.txt"
valid_file_path = "./data/data_validation.txt"
test_file_path = "./data/data_test.txt"
model_name = "EleutherAI/pythia-1b"
output_dir = "./models/pythia-1b-fine-tuned"
lr = 1e-5 #dolly default
batch_size = 4
num_train_epochs = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-1b")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_special_tokens({"additional_special_tokens": [END_KEY, INSTRUCTION_KEY, RESPONSE_KEY_NL]})
model = AutoModelForCausalLM.from_pretrained("EleutherAI/pythia-1b")
model = model.to(device)

data_collator = DataCollatorForCompletionOnlyLM(
    tokenizer=tokenizer, mlm=False, return_tensors="pt", pad_to_multiple_of=batch_size
)

train_dataset = GoalSolutionDataset(tokenizer, file_path=train_file_path, max_length=512)
valid_dataset = GoalSolutionDataset(tokenizer, file_path=valid_file_path, max_length=512)
test_dataset = GoalSolutionDataset(tokenizer, file_path=test_file_path, max_length=512)

c:\Users\shwet\anaconda3\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\shwet\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


## Tokenizer

In [33]:
# See if word is in vocabulary
def word_is_in_vocab(word):
    vocab = tokenizer.get_vocab()
    if word in vocab:
        print(f"The word '{word}' is in the vocabulary.")
    else:
        print(f"The word '{word}' is not in the vocabulary.")

In [11]:
tokenizer.get_vocab()

{'His': 8389,
 'Ġshortage': 28938,
 'Ġbetrayed': 35935,
 'Ġgotta': 17500,
 'ĠAssay': 37140,
 'ĠKob': 43120,
 'ĠBiology': 31958,
 'ĠTina': 45655,
 'abit': 46442,
 'Ġtiempo': 36333,
 'Ġdownhill': 49136,
 'Ġclown': 42493,
 'vet': 14763,
 'Ġ\\*\\**': 38710,
 'WB': 33424,
 '{"': 9819,
 'Ġtruncated': 28069,
 'Ġgenetic': 6380,
 'Ġfn': 21486,
 'Ġrevisit': 45735,
 'imore': 17286,
 'Ġquantify': 22048,
 'ãģ¦ãģĦãĤĭ': 20550,
 'du': 563,
 'arium': 27601,
 'ĠRebecca': 31959,
 'Ġlegend': 13691,
 'medical': 38079,
 'ou': 276,
 'odor': 36155,
 'Ġcarbohydrates': 38401,
 'ĠClick': 15682,
 'asionally': 40650,
 'ĠPoinc': 41327,
 'INSTALL': 42033,
 'ĠResearchers': 43133,
 'LOW': 47123,
 'ĠMeat': 47696,
 'venue': 7290,
 'Ġcoding': 12425,
 'ĠSand': 7889,
 'Ġquark': 19468,
 'ĠHI': 31716,
 'ĠAMPK': 46822,
 'inj': 27497,
 'Ġpathologies': 49404,
 'Ġcensus': 20740,
 'Ġeclipse': 36697,
 'Flor': 31887,
 'Ġglorious': 28548,
 'Ġcircuits': 14174,
 'Ġf': 269,
 'ĠSing': 7712,
 'Ġsang': 21758,
 'aphyl': 25570,
 'FILES': 39

In [13]:
# Size of the complete vocabulary
tokenizer.vocab_size

50254

In [35]:
# Out-of-vocabulary (OOV) Tokens
text = "I want to frabjicate a trinket."
tokenized_text = tokenizer.tokenize(text)
print(tokenized_text)

word_is_in_vocab("frabjicate")

['I', 'Ġwant', 'Ġto', 'Ġfr', 'ab', 'j', 'icate', 'Ġa', 'Ġtr', 'ink', 'et', '.']
The word 'frabjicate' is not in the vocabulary.


The word frabjicate is not in the vocabulary, so the tokenizer splits it into multiple subwords. The accuracy of the token therefore suffers, when it comes to words that don't appear in the vocabulary.

In [30]:
# Word Contractions
text = "Don't worry, it's all good."
tokenized_text = tokenizer.tokenize(text)
print(tokenized_text)

['Don', "'t", 'Ġworry', ',', 'Ġit', "'s", 'Ġall', 'Ġgood', '.']


The tokenizer splits contractions into their constituent parts with the apostroph.

In [39]:
# Special symbols
text = "The # symbol is used for hashtags & $ for dollar."
tokenized_text = tokenizer.tokenize(text)
print(tokenized_text)

word_is_in_vocab("hashtags")

['The', 'Ġ#', 'Ġsymbol', 'Ġis', 'Ġused', 'Ġfor', 'Ġhasht', 'ags', 'Ġ&', 'Ġ$', 'Ġfor', 'Ġdollar', '.']
The word 'hashtags' is not in the vocabulary.


Symbols are tokenized properly as a single token. Again here it is clear, that the word hashtag does not exist in the vocabulary

In [46]:
input_text = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
tokenized_output = tokenizer.tokenize(input_text)
print(tokenized_output)
# Inspect the tokenized_output to ensure that the special tokens (e.g., INSTRUCTION_KEY) are correctly recognized and represented.

['Below', 'Ġis', 'Ġan', 'Ġinstruction', 'Ġthat', 'Ġdescribes', 'Ġa', 'Ġtask', '.', 'ĠWrite', 'Ġa', 'Ġresponse', 'Ġthat', 'Ġappropriately', 'Ġcompletes', 'Ġthe', 'Ġrequest', '.']


In [27]:
t = "How to clean a kitchen? Pleese help me."
print(tokenizer.encode(t))

encoded_t = tokenizer(t)
print(encoded_t)

print(tokenizer.convert_ids_to_tokens(encoded_t['input_ids']))

[2347, 281, 4076, 247, 8576, 32, 21231, 3248, 1361, 479, 15]
{'input_ids': [2347, 281, 4076, 247, 8576, 32, 21231, 3248, 1361, 479, 15], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
['How', 'Ġto', 'Ġclean', 'Ġa', 'Ġkitchen', '?', 'ĠPle', 'ese', 'Ġhelp', 'Ġme', '.']


In [24]:
t = "How to clean a kithen?"
print(tokenizer.encode(t))

encoded_t = tokenizer(t)
print(encoded_t)

print(tokenizer.convert_ids_to_tokens(encoded_t['input_ids']))

[2347, 281, 4076, 247, 6119, 864, 32]
{'input_ids': [2347, 281, 4076, 247, 6119, 864, 32], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}
['How', 'Ġto', 'Ġclean', 'Ġa', 'Ġkit', 'hen', '?']


## GoalSolutionDataset

In [40]:
example = train_dataset[0]
print(example)

{'input_ids': [30003, 310, 271, 9775, 326, 8631, 247, 4836, 15, 19566, 247, 2380, 326, 20420, 29141, 253, 2748, 15, 535, 50278, 187, 6826, 267, 27, 2091, 25317, 10379, 13, 672, 352, 434, 4704, 13, 368, 476, 535, 50279, 37533, 27, 27808, 352, 4830, 247, 5340, 535, 50277, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [7]:
for i, data in enumerate(train_dataset):
    if i < 2:
        print(data)
        original_text = tokenizer.decode(data["input_ids"])
        print(original_text)
        print()

{'input_ids': [30003, 310, 271, 9775, 326, 8631, 247, 4836, 15, 19566, 247, 2380, 326, 20420, 29141, 253, 2748, 15, 535, 50278, 187, 6826, 267, 27, 2091, 25317, 10379, 13, 672, 352, 434, 4704, 13, 368, 476, 535, 50279, 37533, 27, 27808, 352, 4830, 247, 5340, 535, 50277, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

## DataCollator

In [53]:
# Define a sample example with labels
example = [
    {'input_ids': [2347, 281, 4076, 247, 6119, 864, 32], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}
]

# Print the example before data collation
print("Example before data collation:")
print(example)

# Encode the response key token
response_key_token = tokenizer.encode("### Response:\n")

# Modify the data collator to use the response key token
data_collator.response_token_ids = response_key_token

# Call the data collator on the example
batch = data_collator.torch_call(example)

# Print the batch after data collation
print("\nBatch after data collation:")
print(batch)

Example before data collation:
[{'input_ids': [2347, 281, 4076, 247, 6119, 864, 32], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}]


RuntimeError: Could not find response key [50279] in token IDs tensor([2347,  281, 4076,  247, 6119,  864,   32, -100])

## Training

In [8]:
training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    fp16=False,
    bf16=True, # https://www.cerebras.net/machine-learning/to-bfloat-or-not-to-bfloat-that-is-the-question/
    learning_rate=lr,
    logging_dir='./logs/runs',
    logging_strategy="steps",
    logging_steps=10, #dolly defaults
    evaluation_strategy="steps",
    eval_steps=50,
    save_strategy="steps",
    save_steps=400,
    save_total_limit=3,
    load_best_model_at_end=True,
    report_to="wandb",
    remove_unused_columns=False,
    warmup_steps=0,
    #gradient_accumulation_steps=1,
    #gradient_checkpointing=True
)

wandb.init(
    project="NLP-MC2",
    config={
    "learning_rate": lr,
    "architecture": model_name,
    "dataset": "PIQA",
    "epochs": num_train_epochs,
    "batch_size": batch_size,
    "bf16": True,
    "output_dir": output_dir,
    }
)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
)

logger.info("Training...")
trainer.train()

logger.info(f"Saving model to {output_dir}")
trainer.save_model(output_dir)
logger.info("Training complete!")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: jaguar (22hs_i4ds20). Use `wandb login --relogin` to force relogin


/media/bruno/0d2f61d2-2b9c-4043-9a46-8e4dfe74fc95/bruno/anaconda3/envs/nlp-mc2/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/20145 [00:00<?, ?it/s]

You're using a GPTNeoXTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


...

{'eval_loss': 3.2911770343780518, 'eval_runtime': 47.5328, 'eval_samples_per_second': 38.668, 'eval_steps_per_second': 9.678, 'epoch': 4.96}
{'loss': 0.0705, 'learning_rate': 6.701414743112435e-08, 'epoch': 4.97}
{'loss': 0.0426, 'learning_rate': 6.205013651030033e-08, 'epoch': 4.97}
{'loss': 0.0754, 'learning_rate': 5.70861255894763e-08, 'epoch': 4.97}
{'loss': 0.051, 'learning_rate': 5.212211466865228e-08, 'epoch': 4.97}
{'loss': 0.0787, 'learning_rate': 4.715810374782825e-08, 'epoch': 4.98}


  0%|          | 0/460 [00:00<?, ?it/s]

{'eval_loss': 3.291159152984619, 'eval_runtime': 51.4651, 'eval_samples_per_second': 35.714, 'eval_steps_per_second': 8.938, 'epoch': 4.98}
{'loss': 0.0841, 'learning_rate': 4.219409282700422e-08, 'epoch': 4.98}
{'loss': 0.0871, 'learning_rate': 3.72300819061802e-08, 'epoch': 4.98}
{'loss': 0.0558, 'learning_rate': 3.2266070985356175e-08, 'epoch': 4.98}
{'loss': 0.0503, 'learning_rate': 2.7302060064532143e-08, 'epoch': 4.99}
{'loss': 0.0419, 'learning_rate': 2.2338049143708116e-08, 'epoch': 4.99}


  0%|          | 0/460 [00:00<?, ?it/s]

{'eval_loss': 3.290886402130127, 'eval_runtime': 53.2046, 'eval_samples_per_second': 34.546, 'eval_steps_per_second': 8.646, 'epoch': 4.99}
{'loss': 0.058, 'learning_rate': 1.737403822288409e-08, 'epoch': 4.99}
{'loss': 0.0401, 'learning_rate': 1.2410027302060066e-08, 'epoch': 4.99}
{'loss': 0.0499, 'learning_rate': 7.446016381236039e-09, 'epoch': 5.0}
{'loss': 0.0635, 'learning_rate': 2.482005460412013e-09, 'epoch': 5.0}
{'train_runtime': 28429.4477, 'train_samples_per_second': 2.834, 'train_steps_per_second': 0.709, 'train_loss': 0.9632001106985807, 'epoch': 5.0}


In [9]:
model

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50280, 2560)
    (layers): ModuleList(
      (0-31): 32 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (attention): GPTNeoXAttention(
          (rotary_emb): RotaryEmbedding()
          (query_key_value): Linear(in_features=2560, out_features=7680, bias=True)
          (dense): Linear(in_features=2560, out_features=2560, bias=True)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=2560, out_features=10240, bias=True)
          (dense_4h_to_h): Linear(in_features=10240, out_features=2560, bias=True)
          (act): GELUActivation()
        )
      )
    )
    (final_layer_norm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
  )
  (embed_out): Linear(in_features=2560, out_features=50280, bias=False)
)